[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/DragGAN-colab/blob/main/draggan_custom_author_colab.ipynb)

In [ ]:
%cd /content

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install gradio==3.35.2 einops==0.4.1 fused==0.1.0 glfw==2.2.0 imgui==1.3.0 lmdb==1.3.0 mmcv==1.3.15 pypng==0.20220715.0 pyspng==0.1.1 ninja==1.11.1 wandb lpips

!git clone https://github.com/camenduru/PTI
!mkdir /content/PTI/pretrained_models
!wget https://huggingface.co/camenduru/PTI/resolve/main/align.dat -O /content/PTI/pretrained_models/align.dat
!wget https://huggingface.co/camenduru/PTI/resolve/main/ffhq.pkl -O /content/PTI/pretrained_models/ffhq.pkl
!wget https://huggingface.co/camenduru/PTI/resolve/main/e4e_ffhq_encode.pt -O /content/PTI/pretrained_models/e4e_ffhq_encode.pt

%cd /content/PTI

import os
import sys
import pickle
import numpy as np
from PIL import Image
import torch
from configs import paths_config, hyperparameters, global_config
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI
from IPython.display import display
import matplotlib.pyplot as plt
from scripts.latent_editor_wrapper import LatentEditorWrapper

paths_config.input_data_path
image_dir_name = 'image'
use_image_online = True
image_name = 'image'
use_multi_id_training = False
global_config.device = 'cuda'
paths_config.e4e = '/content/PTI/pretrained_models/e4e_ffhq_encode.pt'
paths_config.input_data_id = image_dir_name
paths_config.input_data_path = f'/content/PTI/{image_dir_name}_processed'
paths_config.stylegan2_ada_ffhq = '/content/PTI/pretrained_models/ffhq.pkl'
paths_config.checkpoints_dir = '/content/PTI/'
paths_config.style_clip_pretrained_mappers = '/content/PTI/pretrained_models'
hyperparameters.use_locality_regularization = False

!mkdir /content/PTI/{image_dir_name}_original
!mkdir /content/PTI/{image_dir_name}_processed

In [ ]:
# Please add you image to /content/PTI/image_original/image.png

original_image = Image.open(f'/content/PTI/image_original/{image_name}.jpg')
# original_image
pre_process_images(f'/content/PTI/{image_dir_name}_original')

aligned_image = Image.open(f'/content/PTI/{image_dir_name}_processed/{image_name}.jpeg')
# aligned_image.resize((512,512))

model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

def load_generators(model_id, image_name):
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

  with open(f'{paths_config.checkpoints_dir}/model_{model_id}_{image_name}.pt', 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

  return old_G, new_G

generator_type = paths_config.multi_id_model_type if use_multi_id_training else image_name
old_G, new_G = load_generators(model_id, generator_type)

def export_updated_pickle(new_G,model_id):
  print("Exporting large updated pickle based off new generator and ffhq.pkl")
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda() ## tensor
    old_D = d['D'].eval().requires_grad_(False).cpu()

  tmp = {}
  tmp['G'] = old_G.eval().requires_grad_(False).cpu() # copy.deepcopy(new_G).eval().requires_grad_(False).cpu()
  tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu() # copy.deepcopy(new_G).eval().requires_grad_(False).cpu()
  tmp['D'] = old_D
  tmp['training_set_kwargs'] = None
  tmp['augment_pipe'] = None

  with open(f'{paths_config.checkpoints_dir}/stylegan2_custom_512_pytorch.pkl', 'wb') as f:
      pickle.dump(tmp, f)

export_updated_pickle(new_G,model_id)

In [ ]:
%cd /content
!git clone -b dev1 https://github.com/camenduru/DragGAN
%cd /content/DragGAN
!pip install -r requirements.txt
# !pip install gradio==3.35.2 einops==0.4.1 fused==0.1.0 glfw==2.2.0 imgui==1.3.0 lmdb==1.3.0 mmcv==1.3.15 pypng==0.20220715.0 pyspng==0.1.1 ninja==1.11.1
# !pip install -U scipy==1.11.0 Ninja==1.10.2 gradio>=3.35.2 imageio-ffmpeg>=0.4.3 huggingface_hub hf_transfer pyopengl imgui glfw==2.6.1 pillow>=9.4.0 imageio>=2.9.0 wandb lpips

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2-afhqcat-512x512.pkl -d /content/DragGAN/checkpoints -o stylegan2-afhqcat-512x512.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2-car-config-f.pkl -d /content/DragGAN/checkpoints -o stylegan2-car-config-f.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2-cat-config-f.pkl -d /content/DragGAN/checkpoints -o stylegan2-cat-config-f.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2-ffhq-512x512.pkl -d /content/DragGAN/checkpoints -o stylegan2-ffhq-512x512.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2-lhq-256x256.pkl -d /content/DragGAN/checkpoints -o stylegan2-lhq-256x256.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2_dogs_1024_pytorch.pkl -d /content/DragGAN/checkpoints -o stylegan2_dogs_1024_pytorch.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2_elephants_512_pytorch.pkl -d /content/DragGAN/checkpoints -o stylegan2_elephants_512_pytorch.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2_horses_256_pytorch.pkl -d /content/DragGAN/checkpoints -o stylegan2_horses_256_pytorch.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan2_lions_512_pytorch.pkl -d /content/DragGAN/checkpoints -o stylegan2_lions_512_pytorch.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/DragGAN/resolve/main/stylegan_human_v2_512.pkl -d /content/DragGAN/checkpoints -o stylegan_human_v2_512.pkl

!cp /content/PTI/model_{model_id}_image.pkl /content/DragGAN/checkpoints/model_{model_id}_image.pkl
!python visualizer_drag_gradio.py --share